In [ ]:
data_path = 'D:/workspace/gpt/gptdata/melon_ballad.csv'
model_save = 'D:/workspace/gpt/gptdata/8.발라드가사.csv'
output_path = 'result/발라드가사'
input =  '제주 애월읍 해변위에 앉아 사진을 찍는다.'

# 전처리

In [ ]:
def cleasing(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail주소제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)' # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    text = re.sub(pattern='\n[0-9]', repl='', string=text)
    # pattern = '[^\w\s]' # 특수기호제거
    # text = re.sub(pattern=pattern, repl='', string=text)
    return text    

In [ ]:
import pandas as pd
import numpy as np
import re
df = pd.read_csv(data_path)
df.columns = ['contents']
df = df[['contents']]
df["contents"] = df["contents"].str.replace(pat=r'[^a-zA-Z0-9ㄱ-ㅣ가-힣. ]', repl=r'', regex=True)
df["contents"] = df["contents"].str.replace(pat=r'[\n]', repl=r'', regex=True)
df_new = []
tmp = df['contents']
for i in range(len(tmp)):
    tmp2 = str(tmp[i]).split('.  ')
    tmp2 = list(filter(None, tmp2))
    tmp2 = [item + '.' for item in tmp2]
    df_new.extend(tmp2)
df_new = pd.DataFrame(df_new, columns=['contents'])
df_new

,contents
0,울고 있는 그 여잘 아나요그댈 기다리는 바보 같은 사람멀어져 간단 걸 알면서도그댈 ...
1,그 언젠가 별빛에 새긴 추억 하나가바람에 두고 온 너의 이름을 불러와너의 이름은 떠...
2,언제였죠 그대 내게 처음 오던 날아직도 난 설레는걸요눈이 부시던 그대 모습내 맘 흔...
3,어디까지 말을 할까어떡하면 맘을 알까기다릴 수도 더 버틸 수도 없는무심히 나를 보네...
4,언젠가는 그대를 잊을 때도 오겠지오랜 시간 지나면애써 그댈 지우려 하지는 않아도 되...
...,...
995,오늘일줄야 늘생각은 했지만이른거아냐 좀서운하다네가 했던 따뜻한 말의 온도여기 그대론...
996,집으로 가는 밤길꿈도 잠이 든 이 밤가로등 빛 아래공허하게 날아드는 하루살이몸을 누...
997,내일이 되면널 사랑하게 될 거야예정된 일인 걸불안한 화음들과부딪는 현의 떨림점점 선...
998,You called me your loverYou called me your fri...


In [ ]:
import tqdm
for i in tqdm.tqdm(range(len(df_new))):
    df_new['contents'][i] = cleasing(df_new['contents'][i])
    

100%|██████████| 1000/1000 [00:00<00:00, 20888.59it/s]


In [ ]:
df_new.to_csv(model_save)

# 모델링

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import PreTrainedTokenizerFast

def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer = tokenizer,
        mlm = mlm,
    )
    return data_collator

def train(train_file_path, model_name, output_dir, overwrite_output_dir,
          per_device_train_batch_size, num_train_epochs, save_steps):
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name,
                bos_token = '</s>', eos_token = '</s>', unk_token = '<unk>',
                pad_token = '<pad>', mask_token = '<mask>')
    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)

    tokenizer.save_pretrained(output_dir, legacy_format = False)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    model.save_pretrained(output_dir)

    training_args = TrainingArguments(
        output_dir = output_dir,
        overwrite_output_dir = overwrite_output_dir,
        per_device_eval_batch_size = per_device_train_batch_size,
        num_train_epochs = num_train_epochs,
    )

    trainer = Trainer(
        model = model,
        args = training_args,
        data_collator = data_collator,
        train_dataset = train_dataset,
    )

    trainer.train()
    trainer.save_model()

In [ ]:
model_name = 'skt/kogpt2-base-v2'
output_dir = output_path
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 5.0
save_steps = 500

In [ ]:
train_file_path = model_save
train(train_file_path,
    model_name = model_name,
    output_dir = output_dir,
    overwrite_output_dir = overwrite_output_dir,
    per_device_train_batch_size = per_device_train_batch_size,
    num_train_epochs = num_train_epochs,
    save_steps = save_steps
)

Downloading: 100%|██████████| 2.69M/2.69M [00:01<00:00, 2.12MB/s]
Downloading: 100%|██████████| 0.98k/0.98k [00:00<00:00, 995kB/s]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\transformers\data\datasets\language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Downloading: 100%|██████████| 490M/490M [00:06<00:00, 85.1MB/s]
***** Running training *****
  Num examples = 1749
  Num Epochs = 5
  Instantaneous batch size per de

{'loss': 3.5264, 'learning_rate': 2.71689497716895e-05, 'epoch': 2.28}


Model weights saved in result/발라드가사\checkpoint-500\pytorch_model.bin
 91%|█████████▏| 1000/1095 [01:49<00:09,  9.82it/s]Saving model checkpoint to result/발라드가사\checkpoint-1000
Configuration saved in result/발라드가사\checkpoint-1000\config.json


{'loss': 2.5968, 'learning_rate': 4.337899543378996e-06, 'epoch': 4.57}


Model weights saved in result/발라드가사\checkpoint-1000\pytorch_model.bin
100%|█████████▉| 1094/1095 [02:03<00:00,  9.84it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 1095/1095 [02:04<00:00,  8.82it/s]
Saving model checkpoint to result/발라드가사
Configuration saved in result/발라드가사\config.json


{'train_runtime': 124.3228, 'train_samples_per_second': 70.341, 'train_steps_per_second': 8.808, 'train_loss': 2.9972877397929154, 'epoch': 5.0}


Model weights saved in result/발라드가사\pytorch_model.bin


In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

def load_model(model_path):
  model = GPT2LMHeadModel.from_pretrained(model_path)
  return model

def load_tokenizer(tokenizer_path):
  tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path)
  return tokenizer

def generate_text(sequence, max_lenth):
  model_path =output_path
  model = load_model(model_path)
  tokenizer = load_tokenizer(model_path)
  ids = tokenizer.encode(f'{sequence},', return_tensors = 'pt')
  final_outputs = model.generate(
      ids,
      do_sample = True,
      max_length = max_length,
      pad_token_id = model.config.pad_token_id,
      repetition_penalty = 1.5, # 단어 빈도 패널티
      no_repeat_ngram_size=2, # 반복 줄이기 n개 이상의 토큰이 반복될 경우 등장확률을 0으로 만듬
      temperature=0.9, # 모델의 다음 토큰 확률분포에 변형을 줘 문장을 다양하게 생성
      tok_k = 50, # 가장 높은 확률을 지닌 n개의 단어수 중 추출
      top_p = 0.9 # 누적확률이 n%인 단어까지 포함하여 추출
  )
  result = tokenizer.decode(final_outputs[0], skip_special_tokens = True)
  result = re.sub(pattern='\n[0-9][0-9][0-9][0-9],', repl='', string=result)
  result = re.sub(pattern='\n[0-9][0-9][0-9],', repl='', string=result)
  result = re.sub(pattern='  ', repl=' ', string=result)
  result = re.sub(pattern=',', repl=' ', string=result)
  result = re.sub(pattern='  ', repl=' ', string=result)
  return result

In [ ]:
input = '제주 애월읍 해변위에 앉아 사진을 찍었다.'
sequence = input
max_length = 128 
print('input :' + sequence)
result = generate_text(sequence, max_length)
result

loading configuration file result/발라드가사\config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_i

input :제주 애월읍 해변위에 앉아 사진을 찍었다.


All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at result/발라드가사.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
Didn't find file result/발라드가사\added_tokens.json. We won't load it.
loading file None
loading file result/발라드가사\special_tokens_map.json
loading file result/발라드가사\tokenizer_config.json
loading file result/발라드가사\tokenizer.json


'제주 애월읍 해변위에 앉아 사진을 찍었다. Dont have to goAnd I swear appearing downWhenever you know we hady forget우리가 마주한 이 순간들이어느새 빛나버렸죠 그댄 섬처럼 따뜻해요Yeah Were makes not alright그대는 행복합니다만 우리가 어떻게 우린영원할 수 있습니까.7월의 어느날문득 네 생각이 났었지내가 있는 곳 여기 너가 있어매일같이 날아다니는 수많은 밤'

In [ ]:
output_result = []
for _ in range(5):
    tmp1 = result.strip(input)
    tmp1 = tmp1.strip(', ')
    tmp1 = tmp1.strip(',. ')
    tmp2 = tmp1.split('.')
    input = tmp2[0]+'.'
    input = input.replace('\n', '')
    max_length = 128 
    output_result.append(input)
    globals()['result'] = generate_text(input, max_length)
output_result

loading configuration file result/발라드가사\config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_i

['Dont have to goAnd I swear appearing downWhenever you know we hady forget우리가 마주한 이 순간들이어느새 빛나버렸죠 그댄 섬처럼 따뜻해요Yeah Were makes not alright그대는 행복합니다만 우리가 어떻게 우린영원할 수 있습니까.',
 'ce of loveEven thats into please tell but debtforryHoldings c.',
 'Theres not say youI want to haveMy hand and I knowYour everydrast momentBeautizen the darkAnd fineWere gonna learching foreverspecial onCaused without changelisperish from airSo HighThe view at anciistD.',
 'bluebreakthings insideL.',
 'Im still highThe past is come darkness the worldHere my lifeAnd I special everyday you and i kill findYeah that apart of nightBut gonna tell our ratherWas only foreverEven just stop reachingMemories tooAfterJust alive AwardNobody Originality attempto.']

In [ ]:
input = '제주 애월읍 해변위에 앉아 사진을 찍었다. 행복함.'
sequence = input
max_length = 128 
print('input :' + sequence)
result = generate_text(sequence, max_length).strip(r'[^a-zA-Z0-9ㄱ-ㅣ가-힣. ]')
result = re.sub(pattern='\n[0-9][0-9][0-9][0-9],', repl='', string=result)
result

loading configuration file result/발라드가사\config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_i

input :제주 애월읍 해변위에 앉아 사진을 찍었다. 행복함.


All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at result/발라드가사.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
Didn't find file result/발라드가사\added_tokens.json. We won't load it.
loading file None
loading file result/발라드가사\special_tokens_map.json
loading file result/발라드가사\tokenizer_config.json
loading file result/발라드가사\tokenizer.json


'제주 애월읍 해변위에 앉아 사진을 찍었다. 행복함. 수많은 사진 속에 비친 나의 맘그대 모습 보여줘서미소를 지어도 잊지 말아 줬으면 해.오랜만이야 우린 잊어갔던 우리서로가 서로를 알지 못했던마음의 상처가 이제 너무 깊어진 것 같아나 혼자 남겨진 시간들다 흘려보낸 것들이내게만 남아 더 미웠고너와 함께한 추억이 다 어디로 사라졌는지혼자 남은 기억들로하루 하루 지나보내지도 못한 나였지그래 너 없이 살아갈 수 없어우리가 서로 모를 리 없지니가 떠난 뒤엔 내 사랑들도 전부'

In [ ]:
output_result = []
for i in range(5):
    tmp1 = result.strip(input)
    tmp1 = tmp1.strip(',')
    tmp1 = tmp1.strip(', ')
    tmp1 = tmp1.strip(',. ')
    tmp2 = tmp1.split('.')
    input = tmp2[0]+'.'
    input = input.replace('\n', '')
    max_length = 128 
    output_result.append(input)
    globals()['result'] = generate_text(input, max_length)
output_result

loading configuration file result/발라드가사\config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_i

['수많은 사진 속에 비친 나의 맘그대 모습 보여줘서미소를 지어도 잊지 말아 줬으면 해.',
 '118 7월의 어느날 우연히네게 전화를 걸었는데속삭인다이젠 안녕 전화기 너머로 들려오는너의 목소리는아직도 내 귓가에 생생해너와 함께했던 그때에 난어둠 속을 헤매고 있어오늘 밤만 기다려왔는데이대로 잠들면 어떨까내일 아침에 또 연락올꺼야왜 이런지 몰라답한 마음에 계속 네 연락을 기다리곤 했는지또다시 생각이 나서계속 생각해보니내가 이럴수가 있나 봐끝나지 않는 긴밤.',
 '주치지 않기를그대의 손길도 아직은 서툴렀던 것 같아미안하다는 말.',
 '이 밤 난 너의 곁을 떠나보네.',
 '그 마음처럼모든 게 꿈만 같아좋았던 시간들이 다시 돌아와그땐 참 좋았었는데어디를 둘러봐도 나 혼자만의 착각 속에오늘따라 더 보고 싶어너와 걷고 싶은 그런 나라서네게 기대 어젠밤새 눈물이 흘러끝이 없다는 걸 알아아쉬움에 빠져 또 하루를 보내야 해내일이면 다 끝나겠지늦은 출발이라 후회할 거야나에게도 미련으로 남길 바래잊으려고 애써 볼수록더 아쉬워지는 널 보며혼자 눈물짓던 나를다시는 잡지 말자고하루.']